In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import re
import sys
import time

from matplotlib import image

In [46]:
# Read in image file img1.png
image = cv2.imread('img1.png')
print(image.shape)
#print(image)
print(os.stat('img1.png').st_size)

(546, 656, 3)
425011


In [47]:
# put all pixels into array img_arr
img_arr = []
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        img_arr.append([image[i][j][0], image[i][j][1], image[i][j][2]])

In [4]:
def isprime(n):
    return re.compile(r'^1?$|^(11+)\1+$').match('1' * n) is None
primes = [x for x in range(256) if isprime(x)]

# Adapted from https://newbedev.com/simple-prime-number-generator-in-python

# Base Case Encryption and Decryption

### NOTE: keys are kept minimal to be compatible with float-128, a re-run of this cell may be necessary

In [14]:
print('plaintext:',img_arr[0])
N = 256
# k1 must be co-prime with N
k1 = random.choice(primes)
print('k1:',k1)
k2 = random.randrange(128)
print('k2:',k2)
k3 = random.randrange(128)
print('k3:',k3)
k4 = random.randrange(128)
print('k4:',k4)

plaintext: [116, 139, 153]
k1: 107
k2: 12
k3: 21
k4: 65


In [15]:
e1 = [element * k1 % N for element in img_arr[0]]
print('e1:',e1)
e2 = [element * k2 for element in e1]
print('e2:',e2)
e3 = [element + k3 for element in e2]
print('e3:',e3)
e4 = [element ^ k4 for element in e3]
print('e4:',e4)
npe4 = np.array(e4, dtype=np.float128)
ciphertext = [np.exp(element) for element in npe4]
print('ciphertext:',ciphertext)

e1: [124, 25, 243]
e2: [1488, 300, 2916]
e3: [1509, 321, 2937]
e4: [1444, 256, 2872]
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]


In [16]:
d1 = [int(np.log(element)) for element in ciphertext]
print('d1:',d1)
d2 = [element ^ k4 for element in d1]
print('d2:',d2)
d3 = [element - k3 for element in d2]
print('d3:',d3)
d4 = [int(element/k2) for element in d3]
print('d4:',d4)

d1: [1444, 256, 2872]
d2: [1509, 321, 2937]
d3: [1488, 300, 2916]
d4: [124, 25, 243]


In [17]:
plaintext = [element * pow(k1, -1, N) % N for element in d4]
print('plaintext:',plaintext)

plaintext: [116, 139, 153]


# Full Encryption and Decryption of img1.png

In [48]:
ct = []
i = 0
t0 = time.time()
for px in img_arr:
    e1 = [element * k1 % N for element in px]
    e2 = [element * k2 for element in e1]
    e3 = [element + k3 for element in e2]
    e4 = [element ^ k4 for element in e3]
    npe4 = np.array(e4, dtype=np.float128)
    ciphertext = [np.exp(element) for element in npe4]
    ct.append(ciphertext)
    if (i < 12):
        print("Pixel",i,"iteration")
        print("-------------------")
        print("plaintext:",px)
        print('e1:',e1)
        print('e2:',e2)
        print('e3:',e3)
        print('e4:',e4)
        print('ciphertext:',ciphertext)
        print()
        i += 1
t1 = time.time()
print('Encryption duration:',t1-t0)
print(ct[0:12])

Pixel 0 iteration
-------------------
plaintext: [116, 139, 153]
e1: [124, 25, 243]
e2: [1488, 300, 2916]
e3: [1509, 321, 2937]
e4: [1444, 256, 2872]
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]

Pixel 1 iteration
-------------------
plaintext: [116, 139, 153]
e1: [124, 25, 243]
e2: [1488, 300, 2916]
e3: [1509, 321, 2937]
e4: [1444, 256, 2872]
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]

Pixel 2 iteration
-------------------
plaintext: [116, 139, 153]
e1: [124, 25, 243]
e2: [1488, 300, 2916]
e3: [1509, 321, 2937]
e4: [1444, 256, 2872]
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]

Pixel 3 iteration
-------------------
plaintext: [116, 139, 153]
e1: [124, 25, 243]
e2: [1488, 300, 2916]
e3: [1509, 321, 2937]
e4: [1444, 256, 2872]
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]

Pixel 4 

In [49]:
pt = []
i = 0
t0 = time.time()
for c in ct:
    d1 = [int(np.log(element)) for element in c]
    d2 = [element ^ k4 for element in d1]
    d3 = [element - k3 for element in d2]
    d4 = [int(element/k2) for element in d3]
    plaintext = [element * pow(k1, -1, N) % N for element in d4]
    pt.append(plaintext)
    if (i < 12):
        print("Pixel",i,"iteration")
        print("-------------------")
        print("ciphertext:",c)
        print('d1:',d1)
        print('d2:',d2)
        print('d3:',d3)
        print('d4:',d4)
        print('plaintext:',plaintext)
        print()
        i += 1
t1 = time.time()
print('Decryption duration:',t1-t0)
print(pt[0:12])

Pixel 0 iteration
-------------------
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]
d1: [1444, 256, 2872]
d2: [1509, 321, 2937]
d3: [1488, 300, 2916]
d4: [124, 25, 243]
plaintext: [116, 139, 153]

Pixel 1 iteration
-------------------
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]
d1: [1444, 256, 2872]
d2: [1509, 321, 2937]
d3: [1488, 300, 2916]
d4: [124, 25, 243]
plaintext: [116, 139, 153]

Pixel 2 iteration
-------------------
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]
d1: [1444, 256, 2872]
d2: [1509, 321, 2937]
d3: [1488, 300, 2916]
d4: [124, 25, 243]
plaintext: [116, 139, 153]

Pixel 3 iteration
-------------------
ciphertext: [1.3220012576243031968e+627, 1.5114276650041035426e+111, 1.966762984915421422e+1247]
d1: [1444, 256, 2872]
d2: [1509, 321, 2937]
d3: [1488, 300, 2916]
d4: [124, 25, 243]
plaintext: [116, 139, 153]

Pixel 4 

In [41]:
d_arr = [[[0 for k in range(image.shape[2])] for j in range(image.shape[1])] for i in range(image.shape[0])]
nd_arr = np.asarray(d_arr)
ind = 0
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        nd_arr[i][j] = pt[ind]
        ind += 1
#print(nd_arr)
if ((nd_arr == image).all()):
    print('decrypted image matches original')
cv2.imwrite("img1-decrypted.png", nd_arr)

decrypted image matches original


True

# Full Encryption and Decryption of img2.png

In [50]:
# Read in image file 'img2.png'
image = cv2.imread('img2.png')
print(image.shape)
#print(image)
print(os.path.getsize('img2.png'))

(2048, 1065, 3)
78413


In [51]:
# put all pixels into array img_arr
img_arr = []
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        img_arr.append([image[i][j][0], image[i][j][1], image[i][j][2]])

In [52]:
ct = []
i = 0
t0 = time.time()
for px in img_arr:
    e1 = [element * k1 % N for element in px]
    e2 = [element * k2 for element in e1]
    e3 = [element + k3 for element in e2]
    e4 = [element ^ k4 for element in e3]
    npe4 = np.array(e4, dtype=np.float128)
    ciphertext = [np.exp(element) for element in npe4]
    ct.append(ciphertext)
    if (i < 12):
        print("Pixel",i,"iteration")
        print("-------------------")
        print("plaintext:",px)
        print('e1:',e1)
        print('e2:',e2)
        print('e3:',e3)
        print('e4:',e4)
        print('ciphertext:',ciphertext)
        print()
        i += 1
t1 = time.time()
print('Encryption duration:',t1-t0)
print(ct[0:12])

Pixel 0 iteration
-------------------
plaintext: [117, 143, 159]
e1: [231, 197, 117]
e2: [2772, 2364, 1404]
e3: [2793, 2385, 1425]
e4: [2728, 2320, 1488]
ciphertext: [5.6930714195939167682e+1184, 3.657615217028916652e+1007, 1.6989831513667644326e+646]

Pixel 1 iteration
-------------------
plaintext: [117, 143, 159]
e1: [231, 197, 117]
e2: [2772, 2364, 1404]
e3: [2793, 2385, 1425]
e4: [2728, 2320, 1488]
ciphertext: [5.6930714195939167682e+1184, 3.657615217028916652e+1007, 1.6989831513667644326e+646]

Pixel 2 iteration
-------------------
plaintext: [115, 144, 159]
e1: [17, 48, 117]
e2: [204, 576, 1404]
e3: [225, 597, 1425]
e4: [160, 532, 1488]
ciphertext: [3.0698496406442424668e+69, 1.1083179641103408843e+231, 1.6989831513667644326e+646]

Pixel 3 iteration
-------------------
plaintext: [115, 144, 159]
e1: [17, 48, 117]
e2: [204, 576, 1404]
e3: [225, 597, 1425]
e4: [160, 532, 1488]
ciphertext: [3.0698496406442424668e+69, 1.1083179641103408843e+231, 1.6989831513667644326e+646]

Pixel 4 

In [53]:
pt = []
i = 0
t0 = time.time()
for c in ct:
    d1 = [int(np.log(element)) for element in c]
    d2 = [element ^ k4 for element in d1]
    d3 = [element - k3 for element in d2]
    d4 = [int(element/k2) for element in d3]
    plaintext = [element * pow(k1, -1, N) % N for element in d4]
    pt.append(plaintext)
    if (i < 12):
        print("Pixel",i,"iteration")
        print("-------------------")
        print("ciphertext:",c)
        print('d1:',d1)
        print('d2:',d2)
        print('d3:',d3)
        print('d4:',d4)
        print('plaintext:',plaintext)
        print()
        i += 1
t1 = time.time()
print('Decryption duration:',t1-t0)
print(pt[0:12])

Pixel 0 iteration
-------------------
ciphertext: [5.6930714195939167682e+1184, 3.657615217028916652e+1007, 1.6989831513667644326e+646]
d1: [2728, 2320, 1488]
d2: [2793, 2385, 1425]
d3: [2772, 2364, 1404]
d4: [231, 197, 117]
plaintext: [117, 143, 159]

Pixel 1 iteration
-------------------
ciphertext: [5.6930714195939167682e+1184, 3.657615217028916652e+1007, 1.6989831513667644326e+646]
d1: [2728, 2320, 1488]
d2: [2793, 2385, 1425]
d3: [2772, 2364, 1404]
d4: [231, 197, 117]
plaintext: [117, 143, 159]

Pixel 2 iteration
-------------------
ciphertext: [3.0698496406442424668e+69, 1.1083179641103408843e+231, 1.6989831513667644326e+646]
d1: [160, 532, 1488]
d2: [225, 597, 1425]
d3: [204, 576, 1404]
d4: [17, 48, 117]
plaintext: [115, 144, 159]

Pixel 3 iteration
-------------------
ciphertext: [3.0698496406442424668e+69, 1.1083179641103408843e+231, 1.6989831513667644326e+646]
d1: [160, 532, 1488]
d2: [225, 597, 1425]
d3: [204, 576, 1404]
d4: [17, 48, 117]
plaintext: [115, 144, 159]

Pixel 4 

In [36]:
d_arr = [[[0 for k in range(image.shape[2])] for j in range(image.shape[1])] for i in range(image.shape[0])]
nd_arr = np.asarray(d_arr)
ind = 0
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        nd_arr[i][j] = pt[ind]
        ind += 1
#print(nd_arr)
if ((nd_arr == image).all()):
    print('decrypted image matches original')
cv2.imwrite("img2-decrypted.png", nd_arr)

decrypted image matches original


True

In [ ]:

k1 = random.choice(primes) # get a random prime between 0 and 255
k2 = random.randrange(128) # generate a random integer between 0 and 128
k3 = random.randrange(128)
k4 = random.randrange(128)

for px in img_arr:
    e1 = [element * k1 % N for element in px]
    e2 = [element * k2 for element in e1]
    e3 = [element + k3 for element in e2]
    e4 = [element ^ k4 for element in e3]
    npe4 = np.array(e4, dtype=np.float128)
    ciphertext = [np.exp(element) for element in npe4]
    ct.append(ciphertext)
return ciphertext
    
for c in ct:
    d1 = [int(np.log(element)) for element in c]
    d2 = [element ^ k4 for element in d1]
    d3 = [element - k3 for element in d2]
    d4 = [int(element/k2) for element in d3]
    plaintext = [element * pow(k1, -1, N) % N for element in d4]
    pt.append(plaintext)
return plaintext

